# Project 3
The prediction of molecular properties is an important task in drug discovery. The molecules' atomic composition and arrangement can already tell us a lot about their biological behavior. Each 2D molecule can be represented as a graph, where the nodes are atoms connected by edges corresponding to chemical bonds. The prediction of molecular properties can be formulized as a graph classification task, and graph neural network is usually applied for making graph-level prediction.

In this project, you need develop a model for predicting the toxicity of new molecules. This notebook provides a sample pipeline that establishes a baseline. It is expected that your methods should outperform this baseline. You are strongly encouraged to think about designing more powerful models, finetuning hyperparameters, developing better training strategies, etc.

# Install package

In [1]:
# New these two packages
!pip install torch_geometric
!pip install rdkit-pypi


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Some tutorials.



1.   Pytorch geometric package: https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html
2.   PyTorch Geometric for Graph-Based Molecular Property Prediction using MoleculeNet benchmark: https://medium.com/@nikopavl4/pytorch-geometric-for-graph-based-molecular-property-prediction-using-moleculenet-benchmark-41e36369d3c6
3. Graph neural networks for graph classification. https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing
4. Related github repository on molecular property predictions. https://github.com/yifeiwang15/MotifConv/tree/main/MCM_for_molecule_benchmarks


## What are node and edge features in a molecule.

### Node features:

**Atomic number**: Number of protons in the nucleus of an atom. It’s characteristic of a chemical element and determines its place in the periodic table.

**Chirality**: A molecule is chiral if it is distinguishable from its mirror image by any combination of rotations, translations, and some conformational changes. Different types of chirality exist depending on the molecule and the arrangement of the atoms.

**Degree**: Number of directly-bonded neighbors of the atom.
Formal charge: Charge assigned to an atom. It reflects the electron count associated with the atom compared to the isolated neutral atom.

**Number of H**: Total number of hydrogen atoms on the atom.
Number of radical e: Number of unpaired electrons of the atom.

**Hybridization**: Atom’s hybridization.

**Is aromatic**: Whether it is included in a cyclic structure with pi bonds. This type of structure tends to be very stable in comparison with other geometric arrangements of the same atoms.

**Is in ring**: Whether it is included in a ring (a simple cycle of atoms and bonds in a molecule).

### Edge features:

**Bond type:**: Whether the bond is single, double, triple, or aromatic.

**Stereo Type:** Stereo configuration of the bond.

**Is conjugated**: Whether or not the bond is considered to be conjugated.



# Dataset preparation and train-valid splitting.

In [2]:
import torch
import torch_geometric
import numpy as np
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import pickle

C:\Users\harry\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load datasets. The training and validation sets contain both molecules and their property labels. The test set only contain molecules.
# There are 12 property tasks for prediction. Some properties labels are missing (i.e., nan). You can ignore them.
train_dataset = torch.load("train_data.pt")
valid_dataset = torch.load("valid_data.pt")
test_dataset = torch.load("test_data.pt")

print(f'Size of training set: {len(train_dataset)}')
print(f'Size of validation set: {len(valid_dataset)}')
print(f'Size of test set: {len(test_dataset)}')

Size of training set: 6264
Size of validation set: 783
Size of test set: 784


As we can observe, we have 11 nodes (rows) and each node has 9 features (columns). However, the features provided by Moleculenet are discrete and of type long, so we need to convert them first to continuous embeddings in order to feed them in any ML model.

For example, the first column indicates the atomic number of a node, where 1 represents Hydrogen, 6 represents Carbon, 8 for Oxygen, according to periodic table of elements.

# Build model

In [4]:
# # Atom encoder

# class AtomEncoder(torch.nn.Module):
#     def __init__(self, hidden_channels, num_node_features):
#         super(AtomEncoder, self).__init__()

#         self.embeddings = torch.nn.ModuleList()

#         for i in range(num_node_features):
#             self.embeddings.append(torch.nn.Embedding(100, hidden_channels))

#     def reset_parameters(self):
#         for embedding in self.embeddings:
#             embedding.reset_parameters()

#     def forward(self, x):
#         if x.dim() == 1:
#             x = x.unsqueeze(1)

#         out = 0
#         for i in range(x.size(1)):
#             out += self.embeddings[i](x[:, i])
#         return out


# # A simple graph neural network model

class AtomEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(AtomEncoder, self).__init__()

        self.linear_layers = torch.nn.ModuleList()

        for i in range(num_node_features):
            self.linear_layers.append(torch.nn.Linear(1, hidden_channels))

    def reset_parameters(self):
        for linear in self.linear_layers:
            torch.nn.init.xavier_uniform_(linear.weight)
            torch.nn.init.zeros_(linear.bias)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.linear_layers[i](x[:, i:i+1])
        return out


from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap
import torch.nn.functional as F 
from torch.nn import Linear
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        self.emb = AtomEncoder(hidden_channels=32, num_node_features=num_node_features)
        self.conv1 = GCNConv(hidden_channels,hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, batch):
        x , edge_index, batch_size = batch.x, batch.edge_index, batch.batch
        x = self.emb(x)

        # 2. Readout layer
        x = gap(x, batch_size)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x


## New Changes from here on out
First, we get features that are not included and add them to the original feature datasets.<br>
Below is some testing for each of the Descriptors in the rdkit Chem module

https://www.frontiersin.org/articles/10.3389/fphar.2017.00880/full

In [5]:
from rdkit import Chem
from rdkit.Chem import Descriptors as desc 

print(desc.FpDensityMorgan1(Chem.MolFromSmiles(train_dataset[1].smiles)))
print(desc.FpDensityMorgan2(Chem.MolFromSmiles(train_dataset[1].smiles)))
print(desc.FpDensityMorgan3(Chem.MolFromSmiles(train_dataset[1].smiles)))

0.45
0.7
0.9


In [6]:
descriptor_list = [desc.FpDensityMorgan1,
                   desc.FpDensityMorgan2,
                   desc.FpDensityMorgan3,
                   desc.BalabanJ,
                  desc.BertzCT,
                  desc.Ipc,
                  desc.Kappa1,
                  desc.Kappa2,
                  desc.Kappa3,
                  desc.MolWt,
                  desc.MolLogP,
                  desc.NumRotatableBonds,
                  desc.NumHAcceptors,
                  desc.NumHDonors,
                  desc.NumHeteroatoms,
                  desc.NumValenceElectrons,
                  desc.MolMR,
                  desc.NumRadicalElectrons,
                  desc.RingCount,
                  desc.NumAromaticRings,
                  desc.NumAliphaticRings,
                  desc.NumSaturatedRings,
                  desc.NumAromaticHeterocycles,
                  desc.NumAromaticCarbocycles,
                  desc.NumSaturatedHeterocycles,
                  desc.NumSaturatedCarbocycles,
                  desc.NumAliphaticHeterocycles,
                  desc.NumAliphaticCarbocycles,
                  desc.NOCount,
                  desc.NHOHCount,
                  desc.FractionCSP3,
                  desc.LabuteASA,
                  desc.TPSA,
                   ]

## Make sure none of the descriptors can give NaN as a value.

In [7]:
# Example of preparing data loaders.
# You can use any batch size and see what happens in model performance.

from torch_geometric.data import DataLoader

batch_size=32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

e:\llama\work\.conda\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


# Start training

In [8]:
# train and eval function
from sklearn.metrics import roc_auc_score
def train(model, device, loader, optimizer):
    model.train()

    for step, batch in enumerate(loader):
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(pred.shape).to(torch.float64)

        optimizer.zero_grad()
        ## ignore nan targets (unlabeled) when computing training loss.
        is_labeled = batch.y == batch.y
        loss = criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled]).mean()
        loss.backward()
        optimizer.step()


def eval(model, device, loader):
    model.eval()
    y_true = []
    y_pred = []
    # For every batch in test loader
    for batch in loader:

        batch = batch.to(device)
        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape))
            y_pred.append(pred)

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()
    # Compute the ROC - AUC score and store as history
    rocauc_list = []

    for i in range(y_true.shape[1]):
        #AUC is only defined when there is at least one positive data.
        if np.sum(y_true[:,i] == 1) > 0 and np.sum(y_true[:,i] == 0) > 0:
            # ignore nan values
            is_labeled = y_true[:,i] == y_true[:,i]
            rocauc_list.append(roc_auc_score(y_true[is_labeled,i], y_pred[is_labeled,i]))

    if len(rocauc_list) == 0:
        raise RuntimeError('No positively labeled data available. Cannot compute ROC-AUC.')

    return {'rocauc': sum(rocauc_list)/len(rocauc_list)}


In [9]:
import torch.nn as nn
import torch.optim as optim
import random

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = GCN(32, 10, 12)
optimizer = optim.Adam(model.parameters(), lr=0.01)

def calculate_descriptors(d, descriptor_list):
    dataset = d.copy()
    for graph in dataset:
        mol = Chem.MolFromSmiles(graph.smiles)
        features = []
        for descriptor in descriptor_list:
            value = descriptor(mol)
            features.append(value)
        graph.x = torch.tensor(features).to(torch.float32)
    return dataset

criterion = nn.BCEWithLogitsLoss(reduction="none")

# Loop for multiple iterations
for iteration in range(10):
    print("==== Iteration " + str(iteration + 1))

    # Create copies of the training and validation datasets
    train_dataset_copy = calculate_descriptors(train_dataset, descriptor_list)
    valid_dataset_copy = calculate_descriptors(valid_dataset, descriptor_list)

    # Randomly choose 10 functions from the descriptor_list
    random_functions = random.sample(descriptor_list, 10)

    # Calculate and append descriptors for the new training and validation datasets
    train_dataset_copy = calculate_descriptors(train_dataset_copy, random_functions)
    valid_dataset_copy = calculate_descriptors(valid_dataset_copy, random_functions)

    # Create a new model with the given parameters
    model = GCN(32, 10, 12)
    model.to(device)

    # Start training for 10 epochs
    for epoch in range(1, 11):
        print("Epoch " + str(epoch))

        # Training
        model.train()
        for step, batch in enumerate(train_loader):
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(pred.shape).to(torch.float32)

            optimizer.zero_grad()
            is_labeled = batch.y == batch.y
            loss = criterion(pred.to(torch.float32)[is_labeled], y[is_labeled]).mean()
            loss.backward()
            optimizer.step()

    # Evaluate the final training and validation accuracy
    train_acc = eval(model, device, train_loader)
    val_acc = eval(model, device, val_loader)
    print("Final Training Accuracy:", train_acc)
    print("Final Validation Accuracy:", val_acc)
    print("Functions Used:", [str(func.__name__) for func in random_functions])
    print("====================")



==== Iteration 1


[22:38:28] WARNING: not removing hydrogen atom without neighbors
[22:38:44] WARNING: not removing hydrogen atom without neighbors


Epoch 1


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x320 and 1x32)